In [2]:
account_id_map = {
    'us-east-1': '785573368785',
    'us-east-2': '007439368137',
    'us-west-1': '710691900526',
    'us-west-2': '301217895009',
    'eu-west-1': '802834080501',
    'eu-west-2': '205493899709',
    'eu-west-3': '254080097072',
    'eu-north-1': '601324751636',
    'eu-south-1': '966458181534',
    'eu-central-1': '746233611703',
    'ap-east-1': '110948597952',
    'ap-south-1': '763008648453',
    'ap-northeast-1': '941853720454',
    'ap-northeast-2': '151534178276',
    'ap-southeast-1': '324986816169',
    'ap-southeast-2': '355873309152',
    'cn-northwest-1': '474822919863',
    'cn-north-1': '472730292857',
    'sa-east-1': '756306329178',
    'ca-central-1': '464438896020',
    'me-south-1': '836785723513',
    'af-south-1': '774647643957'
}

In [4]:
!mkdir -p temp_vecs
!cd temp_vecs && wget ftp://ftp.irisa.fr/local/texmex/corpus/gist.tar.gz . && tar -xzvf gist.tar.gz 

--2022-10-05 22:08:10--  ftp://ftp.irisa.fr/local/texmex/corpus/gist.tar.gz
           => ‘gist.tar.gz’
Resolving ftp.irisa.fr (ftp.irisa.fr)... 131.254.254.45, 2001:660:7303:254::45
Connecting to ftp.irisa.fr (ftp.irisa.fr)|131.254.254.45|:21... connected.
Logging in as anonymous ... Logged in!
==> SYST ... done.    ==> PWD ... done.
==> TYPE I ... done.  ==> CWD (1) /local/texmex/corpus ... done.
==> SIZE gist.tar.gz ... 2740172684
==> PASV ... done.    ==> RETR gist.tar.gz ... done.
Length: 2740172684 (2.6G) (unauthoritative)

gist.tar.gz         100%[===================>]   2.55G  21.0MB/s    in 93s     

2022-10-05 22:09:43 (28.2 MB/s) - ‘gist.tar.gz’ saved [2740172684]

--2022-10-05 22:09:43--  http://./
Resolving . (.)... failed: No address associated with hostname.
wget: unable to resolve host address ‘.’
FINISHED --2022-10-05 22:09:43--
Total wall clock time: 1m 34s
Downloaded: 1 files, 2.6G in 1m 33s (28.2 MB/s)


In [12]:
!pip install transformers[torch]
!pip install faiss-cpu

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com, https://pypi.ngc.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 174.0 MB/s eta 0:00:00a 0:00:01


In [4]:
!pip install --upgrade -qU pip awscli boto3 sagemaker transformers

In [14]:
import faiss

In [15]:
import numpy as np

N = 1000000
d = 100
k = 5

# create an array of N d-dimensional vectors (our search space)
xb = np.random.random((N, d)).astype('float32')

In [18]:
import faiss

index = faiss.index_factory(d, "Flat")
index.train(xb)
index.add(xb)

# create a random d-dimensional query vector
xq = np.random.random(d)
distances, neighbors = index.search(xq.reshape(1,-1).astype(np.float32), k)
distances, neighbors

(array([[8.72672 , 8.775642, 8.829514, 8.860796, 9.218237]], dtype=float32),
 array([[283234, 396031, 841282, 170946, 993544]]))

In [19]:
import numpy as np


def ivecs_read(fname):
    a = np.fromfile(fname, dtype="int32")
    d = a[0]
    return a.reshape(-1, d + 1)[:, 1:].copy()


def fvecs_read(fname):
    return ivecs_read(fname).view("float32")

In [21]:
xb = fvecs_read("./temp_vecs/gist/gist_base.fvecs")


In [ ]:
xb.shape[1]

In [22]:
index = faiss.index_factory(xb.shape[1], "IVF4000,Flat")


In [23]:
def cast_object_to_string(data_frame):
    for label in data_frame.columns:
        if data_frame.dtypes[label] == "object":
            data_frame[label] = data_frame[label].astype("str").astype("string")
    return data_frame


In [24]:
batch_size = 100000

index.train(xb[0:batch_size])
faiss.write_index(index, "./models/faiss/trained.index")

n_batches = xb.shape[0] // batch_size
for i in range(n_batches):
    index = faiss.read_index("./models/faiss/trained.index")
    index.add_with_ids(xb[i * batch_size:(i + 1) * batch_size], np.arange(i * batch_size, (i + 1) * batch_size))
    faiss.write_index(index, f"./models/faiss/block_{i}.index")


WARNING clustering 100000 points to 4000 centroids: please provide at least 156000 training points


In [25]:
from faiss.contrib.ondisk import merge_ondisk

index = faiss.read_index("./models/faiss/trained.index")
block_fnames = [f"./models/faiss/block_{b}.index" for b in range(n_batches)]

merge_ondisk(index, block_fnames, "merged_index.ivfdata")
faiss.write_index(index, "./models/faiss/populated.index")


In [27]:
model_faiss = faiss.read_index("./models/faiss/populated.index")


In [28]:
i = 42
model_faiss.make_direct_map()
model_faiss.reconstruct(i).reshape(1,-1).astype(np.float32)


array([[0.1737, 0.1392, 0.1385, 0.0757, 0.0479, 0.1084, 0.1314, 0.0724,
        0.0698, 0.0561, 0.0728, 0.0863, 0.0734, 0.0873, 0.1817, 0.066 ,
        0.1078, 0.0706, 0.1203, 0.1043, 0.0565, 0.0643, 0.0978, 0.0541,
        0.0439, 0.0428, 0.0574, 0.0543, 0.0699, 0.0948, 0.0975, 0.0567,
        0.0504, 0.0411, 0.0581, 0.0777, 0.0418, 0.0452, 0.097 , 0.0447,
        0.0275, 0.0467, 0.0924, 0.0501, 0.0606, 0.0643, 0.0663, 0.0596,
        0.0383, 0.0338, 0.0635, 0.0659, 0.0311, 0.0291, 0.0448, 0.0517,
        0.0241, 0.0373, 0.0895, 0.0853, 0.0597, 0.0505, 0.0547, 0.0568,
        0.0304, 0.0473, 0.052 , 0.0514, 0.0371, 0.036 , 0.0428, 0.0744,
        0.0331, 0.0589, 0.1055, 0.074 , 0.0493, 0.0617, 0.0653, 0.0692,
        0.0332, 0.0633, 0.057 , 0.042 , 0.0337, 0.0593, 0.0638, 0.0696,
        0.0479, 0.0591, 0.0809, 0.0623, 0.0589, 0.0627, 0.0784, 0.0603,
        0.0369, 0.0736, 0.0976, 0.0783, 0.0408, 0.0629, 0.1274, 0.0849,
        0.0702, 0.0979, 0.101 , 0.0614, 0.0591, 0.0633, 0.1427, 

In [29]:
xq = fvecs_read("./temp_vecs/gist/gist_query.fvecs")

index.nprobe = 80
distances, neighbors = model_faiss.search(xq, k)


In [30]:
distances, neighbors

(array([[0.70097166, 0.772008  , 0.78201497, 0.787987  , 0.79519874],
        [0.89665043, 0.9093182 , 0.91346693, 0.935731  , 0.9382586 ],
        [0.40913045, 0.47464633, 0.4957382 , 0.501045  , 0.5051237 ],
        ...,
        [1.3362881 , 1.3506727 , 1.3591287 , 1.360177  , 1.3828452 ],
        [0.8345509 , 0.87380904, 0.89758533, 0.9062674 , 0.9311694 ],
        [0.17919508, 0.17941491, 0.18127467, 0.18271057, 0.18465412]],
       dtype=float32),
 array([[326242, 289394, 413514, 579359, 747797],
        [516068, 373583, 191773, 630858,  17646],
        [242301, 314243, 706382, 199090, 649129],
        ...,
        [127015, 236635,  48954, 518023, 825604],
        [873020, 767512,  40794, 878262, 567044],
        [151863, 602670, 902562, 900252, 900278]]))

### Large Models FAISS

In [54]:
def train_1_epoch(curr_index, train_times):
    batch_size = 100000

    curr_index.train(xb[0:batch_size])
    faiss.write_index(curr_index, "./models/faiss_llm/trained.index")

    n_batches = xb.shape[0] // batch_size
    block_fnames = []
    for epoch_batch in range(train_times):
        print(f"epoch_batch={epoch_batch}::train_times={train_times}::n_batches={n_batches}:xb.shape[0]={xb.shape[0]}::batch_size={batch_size}::" )
        for i in range(n_batches):
            curr_index = faiss.read_index("./models/faiss_llm/trained.index")
            curr_index.add_with_ids(xb[i * batch_size:(i + 1) * batch_size], np.arange(i * batch_size, (i + 1) * batch_size))
            #faiss.write_index(curr_index, f"./models/faiss_llm/block_{i}.index")
            faiss.write_index(curr_index, f"./models/faiss_llm/block_{epoch_batch*10 + i}.index")
            block_fnames.append(f"./models/faiss_llm/block_{epoch_batch*10 + i}.index")

    
    curr_index = faiss.read_index("./models/faiss_llm/trained.index")
    #block_fnames = [f"./models/faiss_llm/block_{b}.index" for b in range(n_batches)]

    merge_ondisk(curr_index, block_fnames, "merged_index.ivfdata")
    faiss.write_index(curr_index, "./models/faiss_llm/populated.index")  
    model_llm_faiss = faiss.read_index("./models/faiss_llm/populated.index")
    return model_llm_faiss

    

In [55]:
model_llm_faiss = faiss.read_index("./models/faiss/populated.index")  
! rm -r ./models/faiss_llm/
!mkdir -p ./models/faiss_llm/

In [ ]:
model_llm_faiss = faiss.index_factory(xb.shape[1], "IVF4000,Flat")
batch_size = 100000

model_llm_faiss.train(xb[0:batch_size])
faiss.write_index(model_llm_faiss, "./models/faiss/trained.index")

train_1_epoch(model_llm_faiss,10)

WARNING clustering 100000 points to 4000 centroids: please provide at least 156000 training points


train_times=10::n_batches=10:xb.shape[0]=1000000::batch_size=100000::
train_times=10::n_batches=10:xb.shape[0]=1000000::batch_size=100000::
train_times=10::n_batches=10:xb.shape[0]=1000000::batch_size=100000::
train_times=10::n_batches=10:xb.shape[0]=1000000::batch_size=100000::
train_times=10::n_batches=10:xb.shape[0]=1000000::batch_size=100000::
train_times=10::n_batches=10:xb.shape[0]=1000000::batch_size=100000::
train_times=10::n_batches=10:xb.shape[0]=1000000::batch_size=100000::
